In [3]:
from time import time
import math, os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from model import Autoencoder
import numpy as np
import collections
from sklearn import metrics
import scanpy as sc
